# La regresion lineal simple  
Para esta practica se utilizara del libro de Econometria de Wooldridge el modelo **"CEO Salary and Return on Equity"**. Donde se propone la siguiente ecuación:

$$Salary = \beta_0 + \beta_1 \cdot ROE + \epsilon$$

Ya que se pretende hacer sin ninguna librería que realice los cálculos matemáticos de forma automático se deben de importar de forma minima las siguientes librerías: *numpy, pandas y matplotlib.pyplot(para realizar las gráficas, no para cálculos)*.

Se deben de obtener los siguientes resultados para poder aceptar la forma en que se realizaron los cálculos

$$\widehat{Salary} = 963.1913 + 18.50119 \cdot ROE + \epsilon$$

Es decir

$$\hat{\beta_0} = 963.1913$$
$$\hat{\beta_1} = 18.50119$$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Para poder descargar los datos de Wooldridge se usara la librería "wooldridge"

In [2]:
import wooldridge as woo

De la siguiente forma se obtiene la base de datos y posteriormente se separan las variables según corresponde

In [3]:
Data = woo.dataWoo("ceosal1")

Data.head()

,salary,pcsalary,sales,roe,pcroe,ros,indus,finance,consprod,utility,lsalary,lsales
0,1095,20,27595.000000,14.1,106.400002,191,1,0,0,0,6.998509,10.225389
1,1001,32,9958.000000,10.9,-30.600000,13,1,0,0,0,6.908755,9.206132
2,1122,9,6125.899902,23.5,-16.299999,14,1,0,0,0,7.022868,8.720281
3,578,-9,16246.000000,5.9,-25.700001,-21,1,0,0,0,6.359574,9.695602
4,1368,7,21783.199219,13.8,-3.000000,56,1,0,0,0,7.221105,9.988894


In [4]:
y = Data["salary"]
y.head()

0    1095
1    1001
2    1122
3     578
4    1368
Name: salary, dtype: int64

In [5]:
x = Data["roe"]
x.head()

0    14.1
1    10.9
2    23.5
3     5.9
4    13.8
Name: roe, dtype: float64

Ahora se determinaran los coeficientes de forma uno a uno, y posteriormente de forma matricial.

$$ \hat{\beta_1} = \frac{\sum(x_i-\bar{x})(y_i-\bar{y})}{\sum(x_i-\bar{x})^2} $$
$$ \hat{\beta_0} = \bar{y} - \beta_1\bar{x} $$

Un punto importante a tomar en cuenta respecto a $\hat{\beta_1}$ es que:

$$\sum(x_i-\bar{x})(y_i-\bar{y}) = COV(x,y)$$

$$\sum(x_i-\bar{x})^2 = VAR(x)$$

$$ \hat{\beta_1} = \frac{COV(x,y)}{VAR(x)}$$

Entonces ya solo queda calcular

$$ \hat{\beta_0} = \bar{y} - \beta_1\bar{x} $$

In [6]:
# La forma simple

COV_XY = np.cov(x,y)[0,1] # Se hace de esta forma ya que 'np.cov' da la matriz var-covar
VAR_X = np.var(x, ddof=1) # ddof es la corrección de grados de libertad

Beta_1 = COV_XY / VAR_X


print(f'Beta_1 = {round(Beta_1,ndigits=5)}')

Beta_1 = 18.50119


In [7]:
y_mean = np.mean(y)
x_mean = np.mean(x)

Beta_0 = y_mean - Beta_1 * x_mean

print(f'Beta_0 = {round(Beta_0,ndigits=4)}')

Beta_0 = 963.1913


In [8]:
# La forma mas intensa

def variance(lst):
    m = sum(lst) / len(lst)
    return sum((xi - m) ** 2 for xi in lst) / (len(lst) - 1)

def covariance(x, y):
    mean_x, mean_y = sum(x) / len(x), sum(y) / len(y)
    cov = sum((x[i] - mean_x) * (y[i] - mean_y) for i in range(len(x)))
    return cov / (len(x) - 1)

COV_XY = covariance(x, y)
VAR_X = variance(x)

Beta_1 = COV_XY / VAR_X
print(f'Beta_1 = {round(Beta_1, ndigits=5)}')

y_mean = sum(y) / len(y)
x_mean = sum(x) / len(x)

Beta_0 = y_mean - Beta_1 * x_mean
print(f'Beta_0 = {round(Beta_0, ndigits=4)}')

Beta_1 = 18.50119
Beta_0 = 963.1913
